# Working with data

1. torch.utils.data.DataLoader
2. torch.utils.data.Dataset

Dataset을 포함한 도메인 별 라이브러리 TorchText, TorchVision, TorchAudio 중 TorchVision의 FashionMNIST dataset 사용

In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [2]:
device = "cpu"
if torch.cuda.is_available():
    deviceNo = torch.cuda.current_device
    print(torch.cuda.get_device_name(0))
    device = "cuda"
print(f"Using {device} device")

NVIDIA GeForce RTX 3060
Using cuda device


# Download training/test data
  from torchvision.datasets.FashionMNIST with transforming to tensor

In [3]:
# Training Data
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

# Test Data
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

# Multiprocess Data Loading

In [4]:
batch_size = 64

# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    r"""
        N: Number of Batch Size
        C: Count of Data Fragment(Height, Width), 1
    """
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


# Creating Models
Neural Network Model을 만들기 위해 **nn.Module** 클래스를 상속받는 클래스를 만든다.

In [5]:
# Model 클래스 생성
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512), # X.shape Height * Width
            nn.ReLU(), # return x if x > 0 else 0
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

# Model 클래스 인스턴스화
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


# Model Parameter 최적화
모델 학습을 위해 loss function과 optimizer 필요

In [6]:
loss_fn = nn.CrossEntropyLoss() # {실제값 * log(예측확률)의 총 합} / 데이터 개수
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3) # Stochastic Gradient Descent 경사하강법, lr = learning rate

# Model Training Function

In [7]:
# 모델 학습 함수

def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train() # set training mode
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # prediction error 계산
        pred = model(X)
        loss = loss_fn(pred, y)

        r"""
            BackPropagation 역전파
            (X, y)의 쌍 한 개의 예측이 끝나면 역전파를 통해 모델의 모든 파라미터 값을 재조정
            모든 (X, y) 쌍의 예측이 끝날 때까지 반복
        """
        loss.backward()

        r"""
            역전파한 오차를 통해 Gradient 계산 후 Parameter 재조정
        """
        optimizer.step()

        r"""
            Parameter 계산 후 필요없는 Gradient 초기화
        """
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f} [{current:>5d}/{size:>5d}]")

# Model Test Function

In [8]:
# model 학습 중 performance test
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval() # set evaluation mode
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f}\n")

# Training & Test

In [9]:
# epochs 수만큼 반복 학습
epochs = 5

for t in range(epochs):
    print(f"Epoch {t+1}\n-----------------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-----------------------------------------
loss: 2.296294 [   64/60000]
loss: 2.280987 [ 6464/60000]
loss: 2.260886 [12864/60000]
loss: 2.253731 [19264/60000]
loss: 2.246687 [25664/60000]
loss: 2.209404 [32064/60000]
loss: 2.218292 [38464/60000]
loss: 2.175638 [44864/60000]
loss: 2.169539 [51264/60000]
loss: 2.138575 [57664/60000]
Test Error: 
 Accuracy: 42.1%, Avg loss: 2.131643

Epoch 2
-----------------------------------------
loss: 2.143432 [   64/60000]
loss: 2.132345 [ 6464/60000]
loss: 2.064896 [12864/60000]
loss: 2.086692 [19264/60000]
loss: 2.041550 [25664/60000]
loss: 1.970677 [32064/60000]
loss: 2.006857 [38464/60000]
loss: 1.906194 [44864/60000]
loss: 1.918386 [51264/60000]
loss: 1.851660 [57664/60000]
Test Error: 
 Accuracy: 57.6%, Avg loss: 1.841541

Epoch 3
-----------------------------------------
loss: 1.875644 [   64/60000]
loss: 1.846940 [ 6464/60000]
loss: 1.714907 [12864/60000]
loss: 1.768369 [19264/60000]
loss: 1.668893 [25664/60000]
loss: 1.612713 [32064/6

# Save Model Parameter

In [10]:
torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")

Saved PyTorch Model State to model.pth


# Load Model

In [11]:
model = NeuralNetwork().to(device)
model.load_state_dict(torch.load("model.pth"))

<All keys matched successfully>

In [12]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    x = x.to(device)
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f"Predicted: \"{predicted}\", Actual: \"{actual}\"")

Predicted: "Ankle boot", Actual: "Ankle boot"
